In [4]:
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


In [5]:

load_dotenv()
SERVICE_KEY = os.getenv('OPEN_API_KEY')

# Cache 디렉토리 지정
cache_dir = LocalFileStore("./.cache/")


llm = ChatOpenAI( openai_api_key= SERVICE_KEY,model_name="gpt-3.5-turbo-1106", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=200,
#     chunk_overlap=50
# )
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("./files/word_test.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings(openai_api_key= SERVICE_KEY)

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)


In [6]:

results = vectorstore.similarity_search("where does winston live")

retriever = vectorstore.as_retriever()

# retriever로부터 documents들을 입력받고, LLM을 통해 추출된 document를 살펴보면서 각 document에 대해 생성된 응답들을 전부 합쳐 한개의 최종 document를 만들고
# 해당 documnet를 final prompt에 입력하여 LLM에 전달 LLM은 질문에 대해 답변해줌.

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


# .content는 ai message (String 값임)
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    results = []
    for doc in documents:
        result = map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        results.append(result)
    "\n\n".join(results)
    return results

#RunnableLambda 는 Chain과 그 내 부 어디서든 function을 호출할수 있도록 해줌.
map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("How many ministries are mentioned")

Victory Mansions is a building with glass doors and a gritty, dusty hallway that smells of boiled cabbage and old rag mats. It has a colored poster of a large face with a caption that reads "BIG BROTHER IS WATCHING YOU." The building has a faulty lift and is seven flights up. Inside the flat, there is a telescreen that cannot be completely shut off.

AIMessageChunk(content='Victory Mansions is a building with glass doors and a gritty, dusty hallway that smells of boiled cabbage and old rag mats. It has a colored poster of a large face with a caption that reads "BIG BROTHER IS WATCHING YOU." The building has a faulty lift and is seven flights up. Inside the flat, there is a telescreen that cannot be completely shut off.')